## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`cs285_f2021/...`). Double click a file to open an editor. There is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

## Run Actor Critic

In [1]:
#@title imports
import os
import time

from cs285.agents.ac_agent import ACAgent
from cs285.infrastructure.rl_trainer import RL_Trainer

d:\SF\Anaconda\envs\rlenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#@title runtime arguments

class ACArgs:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  def __contains__(self, key):
    return hasattr(self, key)

  env_name = 'CartPole-v0' #@param ['CartPole-v0', 'InvertedPendulum-v2', 'HalfCheetah-v2']
  exp_name = 'q4_ac_1_1' #@param

  ## PDF will tell you how to set ep_len
  ## and discount for each environment
  ep_len = 200 #@param {type: "integer"}

  #@markdown batches and steps
  batch_size = 1000 #@param {type: "integer"}
  eval_batch_size =  400#@param {type: "integer"}

  n_iter = 100 #@param {type: "integer"}
  num_agent_train_steps_per_iter = 1 #@param {type: "integer"}
  num_actor_updates_per_agent_update = 1 #@param {type: "integer"}
  num_critic_updates_per_agent_update = 1 #@param {type: "integer"}
  
  #@markdown Actor-Critic parameters
  discount =  0.9#@param {type: "number"}
  learning_rate = 5e-3 #@param {type: "number"}
  dont_standardize_advantages = False #@param {type: "boolean"}
  num_target_updates = 10 #@param {type: "integer"}
  num_grad_steps_per_target_update = 10 #@param {type: "integer"}
  n_layers = 2 #@param {type: "integer"}
  size = 64 #@param {type: "integer"}

  #@markdown system
  save_params = False #@param {type: "boolean"}
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

  #@markdown logging
  ## default is to not log video so
  ## that logs are small enough to be
  ## uploaded to gradscope
  video_log_freq =  -1#@param {type: "integer"}
  scalar_log_freq = 10 #@param {type: "integer"}


args = ACArgs()


if args['video_log_freq'] > 0:
  import warnings
  warnings.warn(
      '''\nLogging videos will make eventfiles too'''
      '''\nlarge for the autograder. Set video_log_freq = -1'''
      '''\nfor the runs you intend to submit.''')

In [3]:
#@title Define AC trainer

class AC_Trainer(object):

    def __init__(self, params):

        #####################
        ## SET AGENT PARAMS
        #####################

        computation_graph_args = {
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
            'num_target_updates': params['num_target_updates'],
            'num_grad_steps_per_target_update': params['num_grad_steps_per_target_update'],
            }

        estimate_advantage_args = {
            'gamma': params['discount'],
            'standardize_advantages': not(params['dont_standardize_advantages']),
        }

        train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
            'num_critic_updates_per_agent_update': params['num_critic_updates_per_agent_update'],
            'num_actor_updates_per_agent_update': params['num_actor_updates_per_agent_update'],
        }

        agent_params = {**computation_graph_args, **estimate_advantage_args, **train_args}

        self.params = params
        self.params['agent_class'] = ACAgent
        self.params['agent_params'] = agent_params
        self.params['train_batch_size'] = params['batch_size']
        self.params['batch_size_initial'] = self.params['batch_size']
        self.params['non_atari_colab_env'] = True

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            self.params['n_iter'],
            collect_policy = self.rl_trainer.agent.actor,
            eval_policy = self.rl_trainer.agent.actor,
            )


In [4]:
#@title create directories for logging


data_path =r'D:\Code\RL-homework\hw3\data'

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

print("LOGGING TO: ", logdir)

LOGGING TO:  D:\Code\RL-homework\hw3\data\q4_ac_1_1_CartPole-v0_06-10-2022_18-42-58


In [5]:
#@title run training
trainer = AC_Trainer(args)
trainer.run_training_loop()

########################
logging outputs to  D:\Code\RL-homework\hw3\data\q4_ac_1_1_CartPole-v0_06-10-2022_18-42-58
########################
Using GPU id 0
Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=1, bias=True)
  (5): Identity()
)


********** Iteration 0 ************


Training agent...

Training agent using sampled data from replay buffer...


d:\SF\Anaconda\envs\rlenv\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([1, 1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



Beginning logging procedure...

Eval_AverageReturn : 28.928571701049805
Eval_StdReturn : 13.935894012451172
Eval_MaxReturn : 71.0
Eval_MinReturn : 13.0
Eval_AverageEpLen : 28.928571428571427
Train_AverageReturn : 27.648649215698242
Train_StdReturn : 12.643160820007324
Train_MaxReturn : 65.0
Train_MinReturn : 11.0
Train_AverageEpLen : 27.64864864864865
Train_EnvstepsSoFar : 1023
TimeSinceStart : 3.2937779426574707
Critic_Loss : 1.2004905939102173
Actor_Loss : -0.0007392740808427334
Initial_DataCollection_AverageReturn : 27.648649215698242
Done logging...




********** Iteration 1 ************


Training agent...

Training agent using sampled data from replay buffer...


********** Iteration 2 ************


Training agent...

Training agent using sampled data from replay buffer...


********** Iteration 3 ************


Training agent...

Training agent using sampled data from replay buffer...


********** Iteration 4 ************


Training agent...

Training agent using sampled data

KeyboardInterrupt: 

In [ ]:
#@markdown You can visualize your runs with tensorboard from within the notebook

## requires tensorflow==2.3.0
# %load_ext tensorboard
# %tensorboard --logdir /content/cs285_f2021/homework_fall2021/hw3/data/